# Import

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import pickle
import torch
import os
import gc
import glob
import scipy.signal
import seaborn as sns
import math
import json
import matplotlib.colors as mcolors
from scipy.signal import butter, filtfilt
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from types import SimpleNamespace
import _MultiResUNet as MultiResUNet
import torch.nn as nn
import torch.optim as optim
import os
import pickle
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt                  
import numpy as np

In [2]:
BATCH_SIZE = 4
EPOCHS = 1000
PREFIX = '20240612_3_'   # model prefix
class_names = ['ramp ascent', 'ramp descent', 'stair ascent', 'stair descent', 'walk']

In [3]:
SAMPLE_RATE = 100  # Hz

In [4]:
def load_config(filename):
    save_path = os.path.join(os.path.join('model', PREFIX), filename)
    with open(save_path, 'r') as f:
        config_dict = json.load(f)
    return SimpleNamespace(**config_dict)

config = load_config(PREFIX + 'config.json')
MODEL_DIR = os.path.join(config.SAVE_DIR, PREFIX)

config.SAVE_DIR = 'FineTune_model'
SAVE_DIR = os.path.join(config.SAVE_DIR, PREFIX)

In [5]:
def save_config(config, save_path):
    os.makedirs(SAVE_DIR, exist_ok=True)
    with open(save_path, 'w') as f:
        json.dump(config.__dict__, f, indent=4)
        
save_config(config, os.path.join(SAVE_DIR, PREFIX + 'config.json'))

In [6]:
# MAX_LENGTH_TARGET를 2 ** model_depth의 배수로 설정
factor = 2 ** config.model_depth
MAX_LENGTH_TARGET = math.ceil((config.SAMPLE_RATE_TARGET * config.MAX_TIME) / factor) * factor
print(f'Max recording time: {MAX_LENGTH_TARGET/config.SAMPLE_RATE_TARGET} sec')

Max recording time: 51.2 sec


# Data Load

In [7]:
def interpolate_time_series(df, sample_rate_target):
    # 시간 열 찾기
    time_col = None
    for col in ['time', 'Time', 'Timestamp(s)', 'Header']:
        if col in df.columns:
            time_col = col
            break
    
    if time_col is None:
        raise ValueError("No time column found in the dataframe")
    
    # 시간 열을 초 단위로 변환
    time = df[time_col].values
    time = time - time[0]  # 시간 축을 0부터 시작하게 변경
    
    # 목표 샘플링 레이트에 따라 새로운 시간축 생성
    duration = time[-1]
    num_samples = int(duration * sample_rate_target)
    new_time = np.linspace(0, duration, num_samples)
    
    # 보간 수행
    interpolated_df = pd.DataFrame({time_col: new_time})
    for col in df.columns:
        if col != time_col and col != 'Label':
            interpolated_df[col] = np.interp(new_time, time, df[col].values)
    
    # 라벨 보간 수행
    if 'Label' in df.columns:
        label_time = df[time_col].values
        labels = df['Label'].values
        label_indices = np.searchsorted(label_time, new_time, side='right') - 1
        label_indices = np.clip(label_indices, 0, len(labels) - 1)
        interpolated_df['Label'] = labels[label_indices]
        
    return interpolated_df

def convert_units(df):
    # 가속도 데이터를 g 단위로 변환 (1 g ≈ 9.81 m/s²)
    accel_columns = [col for col in df.columns if 'Accel' in col]
    for col in accel_columns:
        df[col] = df[col] / 9.81

    # 자이로스코프 데이터를 rad/s 단위로 변환 (1 dps = π/180 rad/s)
    gyro_columns = [col for col in df.columns if 'Gyro' in col]
    for col in gyro_columns:
        df[col] = df[col] * (3.141592653589793 / 180)
    
    return df
# Fine-tuning 데이터 준비
def load_fine_tune_csv_files(fine_tune_folder, sample_rate_target):
    required_columns = [
        'foot_Accel_X', 'foot_Accel_Y', 'foot_Accel_Z', 'foot_Gyro_X', 'foot_Gyro_Y', 'foot_Gyro_Z',
        'shank_Accel_X', 'shank_Accel_Y', 'shank_Accel_Z', 'shank_Gyro_X', 'shank_Gyro_Y', 'shank_Gyro_Z'
        # 'thigh_Accel_X', 'thigh_Accel_Y', 'thigh_Accel_Z', 'thigh_Gyro_X', 'thigh_Gyro_Y', 'thigh_Gyro_Z',
        # 'trunk_Accel_X', 'trunk_Accel_Y', 'trunk_Accel_Z', 'trunk_Gyro_X', 'trunk_Gyro_Y', 'trunk_Gyro_Z'
    ]
    
    csv_files = sorted(glob.glob(os.path.join(fine_tune_folder, '*.csv')))
    X_data = []
    Y_data = []
    
    for file in tqdm(csv_files, desc="Loading Fine-tuning CSV files"):
        df = pd.read_csv(file)
        df = interpolate_time_series(df, sample_rate_target)
        df = convert_units(df)
        X_data.append(df[required_columns])
        Y_data.append(df['Label'].values)
    
    return X_data, Y_data

In [8]:
fine_tune_X_data, fine_tune_Y_data = load_fine_tune_csv_files(config.SAVE_DIR, config.SAMPLE_RATE_TARGET)

Loading Fine-tuning CSV files: 100%|██████████| 20/20 [00:00<00:00, 347.14it/s]


# Feature Engineering

In [9]:
class TimeSeriesFeatureEngineer:
    def __init__(self, window_sizes, sampling_rate):
        self.sampling_rate = sampling_rate
        self.window_sizes = np.dot(window_sizes, sampling_rate).astype(int)
        self.encoder = None
        self.label_mapping = {
            'idle': 'walk',
            'rampascent': 'rampascent',
            'rampascent-walk': 'rampascent',
            'rampdescent': 'rampdescent',
            'rampdescent-walk': 'rampdescent',
            'stairascent': 'stairascent',
            'stairascent-walk': 'stairascent',
            'stairdescent': 'stairdescent',
            'stairdescent-walk': 'stairdescent',
            'stand': 'walk',
            'stand-walk': 'walk',
            'turn1': 'walk',
            'turn2': 'walk',
            'walk': 'walk',
            'walk-rampascent': 'rampascent',
            'walk-rampdescent': 'rampdescent',
            'walk-stairascent': 'stairascent',
            'walk-stairdescent': 'stairdescent',
            'walk-stand': 'walk'
        }

    def map_labels(self, Y_data):
        Y_data_mapped = []
        for y_seq in Y_data:
            Y_data_mapped.append(np.array([self.label_mapping[label] for label in y_seq]))
        return Y_data_mapped

    def create_encoder(self, Y_data):
        # 라벨 매핑
        Y_data_mapped = self.map_labels(Y_data)
        
        # 전체 라벨 수집
        all_labels = np.concatenate(Y_data_mapped)
        all_labels_unique = np.unique(all_labels).reshape(-1, 1)
        
        # OneHotEncoder를 사용하여 라벨 인코딩
        self.encoder = OneHotEncoder(sparse_output=False)
        self.encoder.fit(all_labels_unique)

        # 인코더의 라벨 출력
        print("Encoder classes:", self.encoder.categories_)
        return self.encoder

    def fit_transform_labels(self, Y_data):
        if self.encoder is None:
            raise ValueError("Encoder has not been created. Call create_encoder first.")
        
        # 라벨 매핑
        Y_data_mapped = self.map_labels(Y_data)
        
        # 각 Y_data를 원핫 인코딩
        Y_data_encoded_list = [self.encoder.transform(np.array(y).reshape(-1, 1)) for y in Y_data_mapped]
        return Y_data_encoded_list

    def feature_engineering(self, df: pl.DataFrame):
        # LazyFrame으로 변환하여 작업
        lf = df.lazy()
        
        for col in df.columns:
            # df[col] = self.lowpass_filter(df[col], cutoff_freq=int(self.sampling_rate*0.1), sampling_rate=self.sampling_rate, filter_order=6)

            for window in self.window_sizes:
                window_str = str(window)
                # 통계 값
                lf = lf.with_columns([
                    df[col].rolling_mean(window).alias(col + '_mean_' + window_str),
                    df[col].rolling_std(window).alias(col + '_std_' + window_str),
                    df[col].rolling_min(window).alias(col + '_min_' + window_str),
                    df[col].rolling_max(window).alias(col + '_max_' + window_str),
                    df[col].diff(window).alias(col + '_diff_' + window_str)
                ])
                for lag in [1, 2, 3, 4, 5]:
                    lf = lf.with_columns([
                        df[col].shift(lag * window).alias(col + f'_lag_{lag}_' + window_str)
                    ])
        
        features_df = lf.collect().fill_nan(0).fill_null(0)
        return features_df
    
    def lowpass_filter(self, data, cutoff_freq=100, sampling_rate=200, filter_order=6):
        nyquist = 0.5 * sampling_rate
        normal_cutoff = cutoff_freq / nyquist
        b, a = butter(filter_order, normal_cutoff, btype='low', analog=False)
        filtered_data = filtfilt(b, a, data)
        return filtered_data

    def fit_transform_features(self, X_data):
        X_features = []
        for seq in X_data:
            seq_df = pl.DataFrame(seq)
            features_df = self.feature_engineering(seq_df)
            X_features.append(features_df.to_numpy())
        return X_features

    def resample_x_data(self, X_data, original_sampling_rate, target_sampling_rate):
        resampled_X_data = []
        for seq in X_data:
            # for col in seq.columns:
            #     seq[col] = self.lowpass_filter(seq[col], cutoff_freq=int(self.sampling_rate*0.5), sampling_rate=SAMPLE_RATE, filter_order=6)
                
            num_samples = int(len(seq) * target_sampling_rate / original_sampling_rate)
            resampled_seq = scipy.signal.resample(seq, num_samples)
            resampled_X_data.append(resampled_seq)
        return resampled_X_data

    def resample_y_data(self, Y_data, original_sampling_rate, target_sampling_rate):
        resampled_Y_data = []
        for seq in Y_data:
            num_samples = int(len(seq) * target_sampling_rate / original_sampling_rate)
            resampled_seq = np.zeros((num_samples, seq.shape[1]))
            for i in range(seq.shape[1]):
                resampled_seq[:, i] = np.round(scipy.signal.resample(seq[:, i], num_samples))
            resampled_Y_data.append(resampled_seq)
        return resampled_Y_data

    def fit(self, X_data, Y_data, original_sampling_rate, target_sampling_rate, train_dir="train_batches", val_dir="val_batches", test_size=0.1):
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(val_dir, exist_ok=True)

        # 라벨 인코딩
        self.create_encoder(Y_data)
        Y_data_encoded = self.fit_transform_labels(Y_data)

        # Resample the data
        X_data_resampled = self.resample_x_data(X_data, original_sampling_rate, target_sampling_rate)
        Y_data_resampled = self.resample_y_data(Y_data_encoded, original_sampling_rate, target_sampling_rate)

        # Statistics
        sequence_length = [len(seq) for seq in X_data_resampled]
        print(f'Max sequence length: {max(sequence_length)}')
        print(f'Min sequence length: {min(sequence_length)}')
        print(f'Mean sequence length: {np.mean(sequence_length)}')

        X_features = []
        for idx in range(len(X_data_resampled)):
            X_features.append(self.fit_transform_features([X_data_resampled[idx]])[0])

        if test_size == 0:
            return X_features, [], Y_data_resampled, []
        else:
            # Train/Val split
            X_train, X_val, Y_train, Y_val = train_test_split(X_features, Y_data_resampled, test_size=test_size, random_state=42)
            return X_train, X_val, Y_train, Y_val

In [10]:
feature_engineer = TimeSeriesFeatureEngineer(config.WINDOW_SIZES, config.SAMPLE_RATE_TARGET)

In [11]:
X_train, X_val, Y_train, Y_val = feature_engineer.fit(fine_tune_X_data, fine_tune_Y_data, SAMPLE_RATE, config.SAMPLE_RATE_TARGET, test_size=0.0)

Encoder classes: [array(['rampascent', 'rampdescent', 'stairascent', 'stairdescent', 'walk'],
      dtype='<U12')]
Max sequence length: 3826
Min sequence length: 1028
Mean sequence length: 1761.6


# Dataloader

In [12]:
class FineTuneDataset(Dataset):
    def __init__(self, X_data, Y_data, max_length):
        self.X_data = X_data
        self.Y_data = Y_data
        self.max_length = max_length

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        X_data = self.X_data[idx]
        Y_data = self.Y_data[idx]
        X_padded, X_mask = self.pad_or_trim_sequence(X_data)
        Y_padded, _ = self.pad_or_trim_sequence(Y_data)
        return X_padded, Y_padded, X_mask

    def pad_or_trim_sequence(self, sequence):
        seq_len = len(sequence)
        feature_dim = sequence.shape[1] if len(sequence.shape) > 1 else 1
        if seq_len > self.max_length:
            return torch.tensor(sequence[:self.max_length], dtype=torch.float32), torch.ones(self.max_length, dtype=torch.float32)
        else:
            padding_length = self.max_length - seq_len
            if feature_dim > 1:
                padded_seq = np.pad(sequence, ((0, padding_length), (0, 0)), 'constant', constant_values=0)
            else:
                padded_seq = np.pad(sequence, (0, padding_length), 'constant', constant_values=0)
            mask = np.concatenate([np.ones(seq_len), np.zeros(padding_length)])
            return torch.tensor(padded_seq, dtype=torch.float32), torch.tensor(mask, dtype=torch.float32)

In [13]:
train_dataset = FineTuneDataset(X_train, Y_train, MAX_LENGTH_TARGET)
val_dataset = FineTuneDataset(X_val, Y_val, MAX_LENGTH_TARGET)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print("Fine-tuning DataLoader created successfully.")

Fine-tuning DataLoader created successfully.


In [14]:
# 데이터 로더를 사용하여 모델의 길이, 채널 수 및 출력 채널 수 설정
first_batch = next(iter(train_loader))
length = first_batch[0].shape[1]
num_channel = first_batch[0].shape[2]
output_channels = first_batch[1].shape[-1]

# Training

In [15]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model

In [16]:
model = MultiResUNet.UNet(length=length, model_depth=config.model_depth, num_channel=num_channel, model_width=config.model_width, kernel_size=config.kernel_size, problem_type=config.problem_type, output_channels=output_channels, ds=config.ds, ae=config.ae, feature_number=config.feature_number, is_transconv=config.is_transconv)

criterion = torch.nn.BCEWithLogitsLoss() 

loaded_model = load_model(model, os.path.join(MODEL_DIR, PREFIX+'best_model_checkpoint.pth'))

In [17]:
# 특정 레이어 프리즈 (Deep supervision 레이어와 final convolution 레이어만 학습)
for name, param in model.named_parameters():
    if "deep_supervision" in name or "final_conv" in name or "final_activation" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
    
# 모든 레이어 학습
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)

In [18]:
def fine_tune_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_dir='FineTune_model'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    scaler = GradScaler()

    best_val_loss = float('inf')
    os.makedirs(save_dir, exist_ok=True)

    pbar = tqdm(total=num_epochs, desc="Fine-tuning model", unit="epoch", leave=True)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for X_batch, Y_batch, mask in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            X_batch = X_batch.to(device)
            Y_batch = Y_batch.to(device)
            mask = mask.to(device)

            optimizer.zero_grad()

            with autocast():
                outputs = model(X_batch)
                if isinstance(outputs, list):  # Deep Supervision
                    loss = sum([criterion(output[mask == 1], Y_batch[mask == 1]) for output in outputs]) / mask.sum()
                else:
                    loss = (criterion(outputs, Y_batch) * mask).sum() / mask.sum()

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * X_batch.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        if val_loader:  # Validate if val_loader is not empty
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for X_batch, Y_batch, mask in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}", leave=False):
                    X_batch = X_batch.to(device)
                    Y_batch = Y_batch.to(device)
                    mask = mask.to(device)

                    with autocast():
                        outputs = model(X_batch)
                        if isinstance(outputs, list):  # Deep Supervision
                            loss = sum([criterion(output[mask == 1], Y_batch[mask == 1]) for output in outputs]) / mask.sum()
                        else:
                            loss = (criterion(outputs, Y_batch) * mask).sum() / mask.sum()

                    val_loss += loss.item() * X_batch.size(0)

            val_loss /= len(val_loader.dataset)

            # Save the best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_path = os.path.join(save_dir, PREFIX + 'best_fine_tuned_model_checkpoint.pth')
                torch.save(model.state_dict(), best_model_path)

            pbar.set_postfix({'Train Loss': f'{epoch_loss:.8f}', 'Val Loss': f'{val_loss:.8f}'})
        else:
            # Save the best model
            if epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
                best_model_path = os.path.join(save_dir, PREFIX + 'best_fine_tuned_model_checkpoint.pth')
                torch.save(model.state_dict(), best_model_path)
            pbar.set_postfix({'Train Loss': f'{epoch_loss:.8f}', 'Best Train Loss': f'{best_val_loss:.8f}'})
        pbar.update(1)

    # Save the last model
    last_model_path = os.path.join(save_dir, PREFIX + 'last_fine_tuned_model.pth')
    torch.save(model.state_dict(), last_model_path)

    pbar.close()
    print(f'Finished Fine-tuning. Best validation loss: {best_val_loss:.8f}' if val_loader else 'Finished Fine-tuning.')
    
    return model

In [19]:
# Fine-tune the model
fine_tuned_model = fine_tune_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=EPOCHS, save_dir=SAVE_DIR)

Fine-tuning model:   0%|          | 0/1000 [00:00<?, ?epoch/s]/home/awear-omen/miniforge3/envs/RT5307/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025824022/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
Fine-tuning model: 100%|██████████| 1000/1000 [19:42<00:00,  1.18s/epoch, Train Loss=0.00009175, Best Train Loss=0.00008848]

Finished Fine-tuning.


# Eval

In [20]:
def predict(model, data_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    all_probabilities = []
    running_loss = 0.0
    
    with torch.no_grad():
        for X_batch, Y_batch, mask in data_loader:
            X_batch = X_batch.to(device)
            Y_batch = Y_batch.to(device)
            mask = mask.to(device)

            with autocast():
                outputs = model(X_batch)
                if isinstance(outputs, list):  # Deep Supervision
                    outputs = outputs[-1]  # Use the last output
                loss = criterion(outputs[mask == 1], Y_batch[mask == 1])
                running_loss += loss.item() * X_batch.size(0)

                probs = torch.softmax(outputs, dim=2)  # Calculate probabilities for each class
                probs = probs.cpu().numpy()

                # Apply mask to probabilities
                masked_probs = [probs[j, mask[j].cpu().numpy() == 1] for j in range(probs.shape[0])]

                preds = [np.argmax(p, axis=1) for p in masked_probs]  # Get predicted class indices
                labels = [torch.argmax(Y_batch[j, mask[j] == 1], dim=1).cpu().numpy() for j in range(Y_batch.shape[0])]  # Get true class indices

                all_preds.extend(preds)
                all_labels.extend(labels)
                all_probabilities.extend(masked_probs)
    
    avg_loss = running_loss / len(data_loader.dataset)
    
    return all_preds, all_labels, all_probabilities, avg_loss

In [21]:
def plot_confusion_matrix(true_labels, pred_labels, class_names, save_dir):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig(os.path.join(save_dir, 'confusion_matrix.png'))
    plt.close()

In [22]:
def calculate_accuracy(true_labels, pred_labels):
    accuracy = accuracy_score(true_labels, pred_labels)
    return accuracy

In [23]:
def plot_probabilities(true_labels, pred_labels, probabilities, class_names, save_dir, idx):
    num_classes = len(class_names)
    time_steps = probabilities.shape[0]

    fig, axes = plt.subplots(num_classes, 1, figsize=(10, num_classes * 2), sharex=True)

    if num_classes == 1:
        axes = [axes]

    # Create one-hot encoded true labels
    true_labels_one_hot = np.zeros((time_steps, num_classes))
    for t in range(len(true_labels)):
        true_labels_one_hot[t, true_labels[t]] = 1

    # Create one-hot encoded predicted labels
    pred_labels_one_hot = np.zeros((time_steps, num_classes))
    for t in range(len(pred_labels)):
        pred_labels_one_hot[t, pred_labels[t]] = 1

    color_prob = '#4A4A4A'  # Dark Gray
    color_true = '#00BFFF'  # Deep Sky Blue
    color_pred = '#F08080'  # Light Coral

    for i, class_name in enumerate(class_names):
        axes[i].plot(range(len(probabilities)), probabilities[:, i], label='Probability', alpha=0.6, color=color_prob)
        axes[i].fill_between(range(len(probabilities)), 0, probabilities[:, i], alpha=0.2, color=color_prob)
        axes[i].plot(range(len(true_labels_one_hot)), true_labels_one_hot[:, i], linestyle='dashed', label='True', alpha=0.6, color=color_true)
        axes[i].fill_between(range(len(true_labels_one_hot)), 0, true_labels_one_hot[:, i], alpha=0.2, color=color_true)
        axes[i].plot(range(len(pred_labels_one_hot)), pred_labels_one_hot[:, i], linestyle='dotted', label='Predicted', alpha=0.6, color=color_pred)
        axes[i].fill_between(range(len(pred_labels_one_hot)), 0, pred_labels_one_hot[:, i], alpha=0.2, color=color_pred)
                
        axes[i].set_ylabel('Probability', fontsize=14)
        axes[i].set_ylim(0, 1)
        axes[i].set_title(class_name, fontsize=18)
        axes[i].legend(fontsize=14)

    axes[-1].set_xlabel('Time Steps', fontsize=14)

    fig.suptitle(f'{idx}th Result', fontsize=24, y=0.99, x=0.85)
    plt.tight_layout(rect=[0, 0, 1, 1.02])

    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    save_path = os.path.join(save_dir, f'test_{idx}_probabilities.png')
    plt.savefig(save_path, dpi=300)
    plt.close()

In [24]:
def plot_probabilities_for_all_trials(true_labels, pred_labels, probabilities, class_names, save_dir):
    total_plots = len(probabilities)
    for idx in tqdm(range(total_plots), desc="Plotting probabilities", unit="plot"):
        plot_probabilities(true_labels[idx], pred_labels[idx], probabilities[idx], class_names, save_dir, idx)

In [25]:
model = MultiResUNet.UNet(length=length, model_depth=config.model_depth, num_channel=num_channel, model_width=config.model_width, kernel_size=config.kernel_size, problem_type=config.problem_type, output_channels=output_channels, ds=config.ds, ae=config.ae, feature_number=config.feature_number, is_transconv=config.is_transconv)

criterion = torch.nn.BCEWithLogitsLoss() 

loaded_model = load_model(model, os.path.join(SAVE_DIR, PREFIX+'best_fine_tuned_model_checkpoint.pth'))

In [26]:
data_loader = train_loader

pred_labels, true_labels, probabilities, avg_loss = predict(model, data_loader, criterion)

In [27]:
accuracy = calculate_accuracy(np.concatenate(true_labels).flatten(), np.concatenate(pred_labels).flatten())
print(f"Avg Loss: {avg_loss:.8f}, Accuracy: {accuracy:.8f}")

Avg Loss: 0.61717029, Accuracy: 1.00000000


In [28]:
plot_confusion_matrix(np.concatenate(true_labels).flatten(), np.concatenate(pred_labels).flatten(), class_names, SAVE_DIR)    

In [29]:
plot_probabilities_for_all_trials(true_labels, pred_labels, probabilities, class_names, SAVE_DIR)

Plotting probabilities: 100%|██████████| 20/20 [00:11<00:00,  1.70plot/s]
